In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip,BiSkip

dir_st = 'data/skip-thoughts'
vocab = ['robots', 'are', 'very', 'cool', '<eos>', 'BiDiBu']
biskip = BiSkip(dir_st, vocab)
uniskip = UniSkip(dir_st, vocab)

input = Variable(torch.LongTensor([
    [1,2,3,4,0], # robots are very cool 0
    [5,2,3,0,0]  # bidibu are very cool <eos>
])) # <eos> token is optional

print(input)
def generate_embeding(batch):
    top_half = uniskip(batch)
    bottom_half = biskip(batch)
    combine_skip = torch.cat([top_half,bottom_half],dim=1) 
    return combine_skip
print(generate_embeding(input))
print(generate_embeding(input).size())

/home/grant/Desktop/school/nlp/env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([[1, 2, 3, 4, 0],
        [5, 2, 3, 0, 0]])
tensor([[ 0.1793, -0.1286,  0.0964,  ..., -0.3654, -0.0070, -0.0919],
        [ 0.1372, -0.1739, -0.0526,  ..., -0.1531,  0.0139, -0.1812]],
       grad_fn=<CatBackward>)
torch.Size([2, 4800])


In [2]:
import pandas as pd
from Vocabulary import Vocabulary, preprocess

In [3]:
def make_vocab(tokens):
    voc = Vocabulary(['UNK'])
    voc.add_tokens(tokens)
    print('vocab len is {}'.format(len(voc.w2idx)))
    return voc

In [4]:
def load_data(file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv'):
    '''TODO remove 10% of data for hyper param tuning'''
    df= pd.read_csv(file)
    df = df.drop('InputStoryid',axis=1)
    targets = df['AnswerRightEnding']
    df = df.drop('AnswerRightEnding',axis=1)
    df = df.drop('InputSentence1',axis=1)
    df = df.drop('InputSentence2',axis=1)
    df = df.drop('InputSentence3',axis=1)
    
    voc_str= ''
    for index, row in df.iterrows():
        voc_str+=' '.join(list(row)) + ' '
        
    df['AnswerRightEnding'] = targets
    return df, make_vocab(preprocess(voc_str))

In [5]:
   
df, voc = load_data()


vocab len is 5225


In [6]:
df.head()

,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,I danced terribly and broke a friend's toe.,My friends decided to keep inviting me out as ...,"The next weekend, I was asked to please stay h...",2
1,My eyes were watery and it was hard to breathe.,My allergies were too bad and I had to go back...,It reminded me of how much I loved spring flow...,1
2,She made poor decisions that night and was unf...,Avery thought her children would be happy with...,Avery regretted what she did the next day.,2
3,The eggs his mom used must have been bad though.,Josh thought that the pie was delicious.,Josh got sick.,2
4,"He took a walk, hung out with some friends, an...",He felt inspiration and then went back home to...,John then got an idea for his painting.,1


In [7]:
df['RandomFifthSentenceQuiz1'][0],

('My friends decided to keep inviting me out as I am so much fun.',)

In [8]:
voc.get_sentence(preprocess(df['RandomFifthSentenceQuiz1'][0]))  

[10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 19, 20, 21, 22]

In [9]:
import sys
class LastSentenceDataset(Dataset):
    '''currently implements no context model. will add in last sentence later'''
    def __init__(self,file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv',vocab=None):

        super().__init__()
        df, created_vocab = load_data(file)
        
        if vocab:
            self.vocab = vocab
        else:
            self.vocab = created_vocab
        self.df = df
      
        
        self.dir_st = 'data/skip-thoughts'
        self.biskip = BiSkip(self.dir_st, self.vocab.convert_to_list())
        
        self.uniskip = UniSkip(self.dir_st, self.vocab.convert_to_list())
        
       
        self.data = self.make_data()
        
        
    def __getitem__(self, idx):
        """
        Args:
            idx
        Returns: skip thought embedding of ending and 0/1 if it is the right ending 

        """
        return self.data[idx]

    def __len__(self):
        """
        Returns len of the dataset
        """
        return len(self.data)
       
    def make_data(self):
        data = []
        total = self.df.index
        print('skip thought encoding dataset')
        for i in total:
            #print(row['RandomFifthSentenceQuiz1'],row['RandomFifthSentenceQuiz2'])
           
            progress(i,len(total))
            endings =  self.gen_embbeding(self.df.at[i,'RandomFifthSentenceQuiz1'], self.df.at[i,'RandomFifthSentenceQuiz2'])
            if self.df.at[i,'AnswerRightEnding'] == 1:
                data.append((endings[0],1))
                data.append((endings[1],0))
            else:
                data.append((endings[0],0))
                data.append((endings[1],1))
        return data
    

    def zero_pad(self,l,n):
        l = (l + n * [0])[:n]
        return l
    
    def pad_input(self,a,b):
        ed = sorted([a,b],key=len)
        longer = ed[1]
        shorter = ed[0]
        padded = self.zero_pad( shorter,len(longer))
        if shorter == a:
            return padded,b
        else: return a,padded
        
    def gen_embbeding(self,sent1,sent2):
        try:
            sent1 = preprocess(sent1)
            sent2 = preprocess(sent2)
            #remove random n token that is in one sentence
            if 'n' in sent2:
                sent2.remove('n')
            encoded_end1 = self.vocab.get_sentence(sent1)
            encoded_end2 = self.vocab.get_sentence(sent2)
        except KeyError as e:
            print('key error',e)
            print(sent1,sent2)
        except TypeError as exc:
            print('TYPE ERROR',exc)
            print(sent1,sent2)
        a,b = self.pad_input(encoded_end1,encoded_end2)
        
        batch = Variable(torch.LongTensor([a,b])) 
        top_half = self.uniskip(batch)
        bottom_half = self.biskip(batch)
        combine_skip = torch.cat([top_half,bottom_half],dim=1) 
        return combine_skip
    
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [10]:
#the Dataset constructor or something has a memory leak 
#so repeated execution of this cell can use lots of memory fast. be carefull
l = LastSentenceDataset()
tensor =  l.gen_embbeding('My friends decided to have more fun on time','My friends decided to keep inviting me out as I am so much fun',)
l[0]


vocab len is 5225


/home/grant/Desktop/school/nlp/env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


skip thought encoding dataset


(tensor([-0.1340, -0.2412,  0.1731,  ...,  0.0278, -0.0469,  0.0330],
        grad_fn=<SelectBackward>), 0)

In [11]:
tensor =  l.gen_embbeding('My friends ate','My friends decided to keep inviting me out as I am so much fun')
tensor

tensor([[ 0.0605, -0.1360,  0.0627,  ...,  0.0759,  0.0663,  0.0043],
        [-0.1340, -0.2412,  0.1731,  ...,  0.0278, -0.0469,  0.0330]],
       grad_fn=<CatBackward>)

In [12]:
len(l)

3742

In [13]:
test_data_set = LastSentenceDataset(file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv',vocab=l.vocab)

vocab len is 5225
skip thought encoding dataset


KeyboardInterrupt: 